In [ ]:
import cv2
import mediapipe as mp
import numpy as np
import math
import time

# Initialize MediaPipe Pose detector
mp_pose = mp.solutions.pose
pose = mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5)
mp_drawing = mp.solutions.drawing_utils

# Custom drawing specs
valid_drawing_spec = mp_drawing.DrawingSpec(color=(0, 255, 0), thickness=2, circle_radius=2)  # Green
invalid_drawing_spec = mp_drawing.DrawingSpec(color=(0, 0, 255), thickness=2, circle_radius=2)  # Red

# Load reference images from local files
try:
    ref_img_size = (300, 300)
    tree_pose_img = cv2.imread("tree_pose.jpg")
    downward_dog_img = cv2.imread("downward_dog.jpg")
    goddess_pose_img = cv2.imread("goddess_pose.jpg")
    lotus_pose_img = cv2.imread("lotus_pose.jpg")
    butterfly_pose_img = cv2.imread("butterfly_pose.jpg")
    easy_pose_img = cv2.imread("easy_pose.jpg")
    
    # Resize images
    tree_pose_img = cv2.resize(tree_pose_img, ref_img_size) if tree_pose_img is not None else np.zeros((*ref_img_size, 3), dtype=np.uint8)
    downward_dog_img = cv2.resize(downward_dog_img, ref_img_size) if downward_dog_img is not None else np.zeros((*ref_img_size, 3), dtype=np.uint8)
    goddess_pose_img = cv2.resize(goddess_pose_img, ref_img_size) if goddess_pose_img is not None else np.zeros((*ref_img_size, 3), dtype=np.uint8)
    lotus_pose_img = cv2.resize(lotus_pose_img, ref_img_size) if lotus_pose_img is not None else np.zeros((*ref_img_size, 3), dtype=np.uint8)
    butterfly_pose_img = cv2.resize(butterfly_pose_img, ref_img_size) if butterfly_pose_img is not None else np.zeros((*ref_img_size, 3), dtype=np.uint8)
    easy_pose_img = cv2.resize(easy_pose_img, ref_img_size) if easy_pose_img is not None else np.zeros((*ref_img_size, 3), dtype=np.uint8)
except Exception as e:
    print(f"Error loading images: {e}")
    blank_img = np.zeros((300, 300, 3), dtype=np.uint8)
    tree_pose_img = downward_dog_img = goddess_pose_img = lotus_pose_img = butterfly_pose_img = easy_pose_img = blank_img

# Pose sequence
poses = [
    {"name": "Stand Up", "image": None, "completed": False, "is_transition": True, "message": "Stand Up straight\nGet ready for standing poses"},
    {"name": "Tree Pose", "image": tree_pose_img, "completed": False},
    {"name": "Downward Dog Pose", "image": downward_dog_img, "completed": False},
    {"name": "Goddess Pose", "image": goddess_pose_img, "completed": False},
    {"name": "Sit Down", "image": None, "completed": False, "is_transition": True, "message": "Sit Down comfortably\nGet ready for seated poses"},
    {"name": "Lotus Pose", "image": lotus_pose_img, "completed": False},
    {"name": "Butterfly Pose", "image": butterfly_pose_img, "completed": False},
    {"name": "Easy Pose", "image": easy_pose_img, "completed": False, "is_meditation": True}
]

# State variables
current_pose_index = 0
round_number = 1
show_reference = True
reference_start_time = time.time()
feedback_message = ""
feedback_timer = 0
pose_validated = False
pose_hold_start = 0
pose_hold_duration = 3.0  # Increased hold time to 3 seconds
transition_delay = 1.0
meditation_start = 0
meditation_duration = 60
breath_cycle = "Breathe In"
last_breath_change = 0

def calculate_angle(a, b, c):
    a = np.array([a.x, a.y])
    b = np.array([b.x, b.y])
    c = np.array([c.x, c.y])
    ba = a - b
    bc = c - b
    cosine_angle = np.dot(ba, bc) / (np.linalg.norm(ba) * np.linalg.norm(bc))
    angle = np.degrees(np.arccos(cosine_angle))
    return angle

def check_pose(landmarks, target_pose):
    feedback = []
    all_correct = True
    
    # Get landmarks
    nose = landmarks[mp_pose.PoseLandmark.NOSE.value]
    left_shoulder = landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value]
    right_shoulder = landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value]
    left_elbow = landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value]
    right_elbow = landmarks[mp_pose.PoseLandmark.RIGHT_ELBOW.value]
    left_wrist = landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value]
    right_wrist = landmarks[mp_pose.PoseLandmark.RIGHT_WRIST.value]
    left_hip = landmarks[mp_pose.PoseLandmark.LEFT_HIP.value]
    right_hip = landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value]
    left_knee = landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value]
    right_knee = landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value]
    left_ankle = landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value]
    right_ankle = landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value]

    # Calculate angles
    left_knee_angle = calculate_angle(left_hip, left_knee, left_ankle)
    right_knee_angle = calculate_angle(right_hip, right_knee, right_ankle)
    left_elbow_angle = calculate_angle(left_shoulder, left_elbow, left_wrist)
    right_elbow_angle = calculate_angle(right_shoulder, right_elbow, right_wrist)

    if target_pose == "Stand Up":
        standing = (left_hip.y < left_shoulder.y + 0.1) and (right_hip.y < right_shoulder.y + 0.1)
        if not standing:
            feedback.append("Stand up straight")
            all_correct = False

    elif target_pose == "Tree Pose":
        foot_on_thigh = ((left_ankle.y < left_knee.y and abs(left_ankle.x - right_hip.x) < 0.15)) or \
                        (right_ankle.y < right_knee.y and abs(right_ankle.x - left_hip.x) < 0.15)
        if not foot_on_thigh:
            feedback.append("Place foot on opposite thigh")
            all_correct = False
        
        hands_together = (abs(left_wrist.x - right_wrist.x) < 0.1 and 
                         abs(left_wrist.y - right_wrist.y) < 0.1 and 
                         left_wrist.y < nose.y)
        if not hands_together:
            feedback.append("Bring hands together above head")
            all_correct = False

    elif target_pose == "Downward Dog Pose":
        hips_raised = (left_hip.y < left_shoulder.y) and (right_hip.y < right_shoulder.y)
        if not hips_raised:
            feedback.append("Raise your hips higher")
            all_correct = False
        
        legs_straight = (left_knee_angle > 160) and (right_knee_angle > 160)
        if not legs_straight:
            feedback.append("Straighten your legs")
            all_correct = False

    elif target_pose == "Goddess Pose":
        knees_bent = (left_knee_angle < 120) and (right_knee_angle < 120)
        knees_wide = abs(left_knee.x - right_knee.x) > abs(left_hip.x - right_hip.x)
        if not knees_bent:
            feedback.append("Bend your knees more")
            all_correct = False
        if not knees_wide:
            feedback.append("Widen your stance")
            all_correct = False
        
        arms_bent = (left_elbow_angle < 90) and (right_elbow_angle < 90)
        arms_height = abs(left_wrist.y - left_shoulder.y) < 0.15
        if not arms_bent:
            feedback.append("Bend your elbows")
            all_correct = False
        if not arms_height:
            feedback.append("Raise arms to shoulder height")
            all_correct = False

    elif target_pose == "Sit Down":
        sitting = (left_hip.y > left_knee.y) and (right_hip.y > right_knee.y)
        if not sitting:
            feedback.append("Sit down comfortably")
            all_correct = False

    elif target_pose == "Lotus Pose":
        left_foot_on_right_thigh = (left_ankle.y < right_knee.y and abs(left_ankle.x - right_hip.x) < 0.15)
        right_foot_on_left_thigh = (right_ankle.y < left_knee.y and abs(right_ankle.x - left_hip.x) < 0.15)
        
        if not (left_foot_on_right_thigh or right_foot_on_left_thigh):
            feedback.append("Place at least one foot on opposite thigh")
            all_correct = False
            
        spine_straight = (abs(left_shoulder.x - left_hip.x) < 0.1) and (abs(right_shoulder.x - right_hip.x) < 0.1)
        if not spine_straight:
            feedback.append("Keep spine straight")
            all_correct = False
            
        hands_on_knees = ((abs(left_wrist.x - left_knee.x) < 0.15 and abs(left_wrist.y - left_knee.y) < 0.15) and 
                         (abs(right_wrist.x - right_knee.x) < 0.15 and abs(right_wrist.y - right_knee.y) < 0.15))
        
        hands_prayer = (abs(left_wrist.x - right_wrist.x) < 0.1 and abs(left_wrist.y - right_wrist.y) < 0.1)
        
        if not (hands_on_knees or hands_prayer):
            feedback.append("Place hands on knees or in prayer position")
            all_correct = False

    elif target_pose == "Butterfly Pose":
        knees_bent = (left_knee_angle < 90) and (right_knee_angle < 90)
        feet_together = abs(left_ankle.x - right_ankle.x) < 0.1
        
        if not knees_bent:
            feedback.append("Bend your knees more")
            all_correct = False
        if not feet_together:
            feedback.append("Bring feet together")
            all_correct = False
            
        hands_on_feet = ((abs(left_wrist.x - left_ankle.x) < 0.15 and abs(left_wrist.y - left_ankle.y) < 0.15)) and \
                       ((abs(right_wrist.x - right_ankle.x) < 0.15 and abs(right_wrist.y - right_ankle.y) < 0.15))
        
        if not hands_on_feet:
            feedback.append("Hold your feet with hands")
            all_correct = False

    elif target_pose == "Easy Pose":
        legs_crossed = (left_knee.x > right_ankle.x) and (right_knee.x < left_ankle.x)
        if not legs_crossed:
            feedback.append("Cross your legs comfortably")
            all_correct = False
            
        spine_straight = (abs(left_shoulder.x - left_hip.x) < 0.1) and (abs(right_shoulder.x - right_hip.x) < 0.1)
        if not spine_straight:
            feedback.append("Keep spine straight")
            all_correct = False

    return all_correct, feedback

# Main loop
cap = cv2.VideoCapture(0)
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    frame = cv2.flip(frame, 1)
    current_pose = poses[current_pose_index]
    
    # Special handling for transition poses
    if 'is_transition' in current_pose and current_pose['is_transition']:
        if show_reference:
            # Create a clean frame for transition message
            transition_frame = np.zeros_like(frame)
            cv2.putText(transition_frame, current_pose['name'].upper(), 
                       (frame.shape[1]//2-100, frame.shape[0]//2-50), 
                       cv2.FONT_HERSHEY_SIMPLEX, 1.5, (0, 255, 255), 3)
            
            # Split message into lines if it contains newline characters
            message_lines = current_pose['message'].split('\n')
            for i, line in enumerate(message_lines):
                cv2.putText(transition_frame, line, 
                           (frame.shape[1]//2-200, frame.shape[0]//2 + 30 + i*40), 
                           cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 255), 2)
            
            if time.time() - reference_start_time > 3:  # Show transition for 3 seconds
                show_reference = False
                pose_validated = True
                current_pose['completed'] = True
                current_pose_index = (current_pose_index + 1) % len(poses)
                show_reference = True
                reference_start_time = time.time()
            
            cv2.imshow("Yoga Pose Detection", transition_frame)
            cv2.waitKey(1)
            continue
    
    # Show reference image first
    if show_reference:
        if current_pose['image'] is not None:
            # Create a clean frame for reference image
            ref_frame = np.zeros_like(frame)
            cv2.putText(ref_frame, f"ROUND {round_number}: Match this pose", (50, 50), 
                       cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 255), 2)
            cv2.putText(ref_frame, current_pose['name'], (50, 100), 
                       cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 255), 2)
            
            # Center reference image
            h, w = current_pose['image'].shape[:2]
            x = (frame.shape[1] - w) // 2
            y = (frame.shape[0] - h) // 2
            ref_frame[y:y+h, x:x+w] = current_pose['image']
        else:
            ref_frame = np.zeros_like(frame)
            cv2.putText(ref_frame, current_pose['name'], (frame.shape[1]//2-100, frame.shape[0]//2), 
                       cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 255), 2)
        
        if time.time() - reference_start_time > 2:  # Show reference for 2 seconds
            show_reference = False
            pose_validated = False
            pose_hold_start = 0
            if 'is_meditation' in current_pose and current_pose['is_meditation']:
                meditation_start = time.time()
                last_breath_change = time.time()
        
        cv2.imshow("Yoga Pose Detection", ref_frame)
        cv2.waitKey(1)
        continue
    
    # Process pose detection
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = pose.process(rgb_frame)
    
    pose_valid = False
    feedback = []
    if results.pose_landmarks:
        landmarks = results.pose_landmarks.landmark
        pose_valid, feedback = check_pose(landmarks, current_pose['name'])
        
        # Draw landmarks with appropriate color
        if pose_valid:
            mp_drawing.draw_landmarks(
                frame, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                valid_drawing_spec, valid_drawing_spec
            )
            # Start or continue hold timer
            if pose_hold_start == 0:
                pose_hold_start = time.time()
        else:
            mp_drawing.draw_landmarks(
                frame, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                invalid_drawing_spec, invalid_drawing_spec
            )
            pose_hold_start = 0
            pose_validated = False

    # Special meditation handling for Easy Pose
    if 'is_meditation' in current_pose and current_pose['is_meditation'] and pose_valid:
        current_time = time.time()
        meditation_elapsed = current_time - meditation_start
        remaining_time = max(0, meditation_duration - meditation_elapsed)
        
        # Change breath message every 3 seconds
        if current_time - last_breath_change > 3:
            breath_cycle = "Breathe Out" if breath_cycle == "Breathe In" else "Breathe In"
            last_breath_change = current_time
        
        # Draw meditation timer
        cv2.rectangle(frame, (50, 70), (int(50 + (frame.shape[1]-100) * (meditation_elapsed/meditation_duration)), 90), (0, 255, 255), -1)
        cv2.rectangle(frame, (50, 70), (frame.shape[1]-50, 90), (255, 255, 255), 1)
        cv2.putText(frame, f"Meditation: {int(remaining_time)}s remaining", (50, 65), 
                   cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)
        
        # Draw breath instruction in center
        cv2.putText(frame, breath_cycle, (frame.shape[1]//2-80, frame.shape[0]//2), 
                   cv2.FONT_HERSHEY_SIMPLEX, 1.5, (0, 200, 255), 3)
        
        # Complete meditation after duration
        if meditation_elapsed >= meditation_duration:
            pose_validated = True
            current_pose['completed'] = True
            feedback_message = "Meditation Complete!"
            feedback_timer = time.time()
            
            # Check if all poses completed
            if all(p['completed'] for p in poses):
                # Show completion message
                completion_frame = np.zeros_like(frame)
                cv2.putText(completion_frame, "FANTASTIC!", (frame.shape[1]//2-150, frame.shape[0]//2-50), 
                           cv2.FONT_HERSHEY_SIMPLEX, 1.5, (0, 255, 255), 3)
                cv2.putText(completion_frame, f"You've completed Round {round_number}!", 
                           (frame.shape[1]//2-250, frame.shape[0]//2+20), 
                           cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
                cv2.putText(completion_frame, "Get ready for Round 2!", 
                           (frame.shape[1]//2-200, frame.shape[0]//2+70), 
                           cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
                cv2.imshow("Yoga Pose Detection", completion_frame)
                cv2.waitKey(3000)
                
                # Reset for next round
                round_number += 1
                current_pose_index = 0
                show_reference = True
                reference_start_time = time.time()
                for p in poses:
                    p['completed'] = False
                continue
    else:
        # Normal pose validation and transition logic
        if pose_valid and not pose_validated:
            if pose_hold_start > 0 and (time.time() - pose_hold_start) >= pose_hold_duration:
                feedback_message = "VALID POSE!"
                feedback_timer = time.time()
                pose_validated = True
                current_pose['completed'] = True
                
                time.sleep(transition_delay)
                
                # Move to next pose
                current_pose_index = (current_pose_index + 1) % len(poses)
                show_reference = True
                reference_start_time = time.time()
                pose_validated = False
                
                cv2.imshow("Yoga Pose Detection", frame)
                cv2.waitKey(1)
                continue
        elif not pose_valid:
            if feedback:
                feedback_message = "INVALID: " + " | ".join(feedback[:2])
                feedback_timer = time.time()

    # Display current pose and status
    status = "VALID" if pose_validated else ("HOLDING" if pose_valid and pose_hold_start > 0 else "INVALID")
    text_color = (0, 255, 0) if status == "VALID" else (0, 165, 255) if status == "HOLDING" else (0, 0, 255)
    
    cv2.putText(frame, f"{current_pose['name']}: {status}", (50, 40), 
               cv2.FONT_HERSHEY_SIMPLEX, 0.7, text_color, 2)
    
    # Show round number
    cv2.putText(frame, f"Round: {round_number}", (frame.shape[1] - 150, 40), 
               cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)
    
    # Show feedback message
    if time.time() - feedback_timer < 2 and feedback_message:
        cv2.putText(frame, feedback_message, (50, frame.shape[0] - 30), 
                   cv2.FONT_HERSHEY_SIMPLEX, 0.6, text_color, 1)
    
    # Show pose hold progress if validating (not for meditation)
    if pose_valid and not pose_validated and pose_hold_start > 0 and not ('is_meditation' in current_pose and current_pose['is_meditation']):
        progress = min(1.0, (time.time() - pose_hold_start) / pose_hold_duration)
        cv2.rectangle(frame, (50, 70), (int(50 + 200 * progress), 90), (0, 255, 0), -1)
        cv2.rectangle(frame, (50, 70), (250, 90), (255, 255, 255), 1)
        cv2.putText(frame, "Hold pose...", (50, 65), 
                   cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)
    
    # Show small reference in corner (if available)
    if current_pose['image'] is not None:
        ref_img = cv2.resize(current_pose['image'], (150, 150))
        frame[10:160, frame.shape[1]-160:frame.shape[1]-10] = ref_img
    
    cv2.imshow("Yoga Pose Detection", frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

KeyboardInterrupt: 

: 